## Setup

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from datetime import timedelta
import random

In [2]:
# Cambiamos el directorio para importar las funciones de func_aux
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).absolute().parent))

In [3]:
from src.funcs_aux import *

In [4]:
datos_unidos = pd.read_csv("../data/procesados/datos_unidos.csv")

In [5]:
datos_unidos.columns

Index(['SKU', 'DATE', 'STORE_ID', 'PRICE', 'QUANTITY', 'TOTAL_SALES', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE', 'OPENDATE', 'CLOSEDATE',
       'STORE_SUBGROUP_DATE_ID', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', 'INITIAL_TICKET_PRICE', 'BASE_PRICE',
       'COSTOS', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE',
       'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK'],
      dtype='object')

In [6]:
cols_categoricas = ['SKU', 'STORE_ID', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE',  'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', "DAY_OF_WEEK"]

target = "TOTAL_SALES"

In [7]:
for col in cols_categoricas:
    datos_unidos[col] = datos_unidos[col].astype("category")

In [8]:
datos_unidos.sort_values(by=["DATE", "STORE_ID", "SKU"], inplace=True)

## Feature aggregation

Primero, haremos el promedio de ventas por SKU X STORE_ID de los últimos 7, 30 y 90 días, para compensar el hecho de que no es posible (por limitaciones computacionales) completar todo el dataset con los días en que no hubo transacciones de un producto.

De esta manera, el modelo podrá dilucidar las épocas en donde no hay ventas de ciertos productos.

In [ ]:
# datos_unidos = rolling_sales_completo(datos_unidos)

Aclaración: finalmente, en los modelos no utilizamos estos últimos features dado que elevaban el costo computacional y los modelos no presentaban una mejora significativa de performance con ellos

Ahora, haremos el promedio y desviación estándar de las ventas por subgrupo y por categoría, de manera que el modelo pueda entender mejor los cambios de ventas por épocas del año de grupos más grandes de productos.

In [ ]:
rolling_sales(datos_unidos, group="SUBGROUP", windows=[30, 90], std=False)
rolling_sales(datos_unidos, group="SKU", windows=[30, 90], std=False)
rolling_sales(datos_unidos, group="STORE_ID", windows=[30, 90], std=False)

In [ ]:
datos_unidos = rolling_price_pct(datos_unidos=datos_unidos, group="SKU", windows=[30,90], std=False)
datos_unidos = rolling_price_pct(datos_unidos=datos_unidos, group="SUBGROUP", windows=[30,90], std=False)

## Test de modelos

### LightGBM

In [11]:
import lightgbm as lgb

In [12]:
def walk_forward_lightgbm(df, features, target_col, date_col, categorical_cols,
                          train_days=365, step_days=30, forecast_days=7,
                          params=None):
    """
    df: DataFrame con features + target
    target_col: nombre de la columna objetivo (ej. 'TOTAL_SALES')
    date_col: columna con la fecha
    categorical_cols: lista de columnas categóricas (deben ser dtype 'category')
    train_days, step_days, forecast_days: enteros en días
    params: dict de parámetros LightGBM
    """

    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col)

    results = []
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    start_train_end = min_date + timedelta(days=train_days)

    count = 0

    while start_train_end + timedelta(days=forecast_days) <= max_date:
        count+=1

        print(f"Walk-forward: iteracion numero {count}")

        # Train y Test
        train_data = df[df[date_col] < start_train_end]
        test_data = df[(df[date_col] >= start_train_end) &
                       (df[date_col] < start_train_end + timedelta(days=forecast_days))]

        if len(test_data) == 0:
            break

        # Creamos un validation set para early stopping
        valid_days_inner = 7
        train_end_inner = train_data["DATE"].max() - timedelta(days=valid_days_inner)

        train_inner = train_data[train_data["DATE"] <= train_end_inner]
        valid_inner = train_data[train_data["DATE"] > train_end_inner]

        X_train_inner = train_inner[features]
        y_train_inner = train_inner[target_col]
        X_valid_inner = valid_inner[features]
        y_valid_inner = valid_inner[target_col]

        # Dataset LightGBM
        lgb_train = lgb.Dataset(X_train_inner, label=y_train_inner, categorical_feature=categorical_cols)
        lgb_valid = lgb.Dataset(X_valid_inner, label=y_valid_inner, categorical_feature=categorical_cols, reference=lgb_train)

        model = lgb.train(
            params,
            lgb_train,
            valid_sets=[lgb_train, lgb_valid],
            valid_names=["train_inner", "valid_inner"]
        )

        # Predicciones
        y_test_pred = model.predict(test_data[features], num_iteration=model.best_iteration)
        y_train_pred = model.predict(X_train_inner,num_iteration=model.best_iteration)
        
        # Métricas
        r2_test = r2_score(test_data[target], y_test_pred)
        r2_train = r2_score(y_train_inner, y_train_pred)


        results.append({
            "train_end_date": start_train_end,
            "r2_train": r2_train,
            "r2_test": r2_test
        })

        start_train_end += timedelta(days=step_days)

    return pd.DataFrame(results)

In [13]:
cols = list(datos_unidos.columns)
features = [col for col in cols if col not in ["DATE", "TOTAL_SALES",
                                               'INITIAL_TICKET_PRICE', 'BASE_PRICE', "COSTOS", 
                                               "OPENDATE", "CLOSEDATE", "QUANTITY", "STORE_SUBGROUP_DATE_ID"] ]

print(features)

['SKU', 'STORE_ID', 'PRICE', 'REGION', 'CITY', 'STATE', 'STORE_TYPE', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE', 'PRICE_GROUP_ID', 'BRAND', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK', 'SUBGROUP_mean_30D', 'SUBGROUP_std_30D', 'SUBGROUP_mean_90D', 'SUBGROUP_std_90D', 'SUBGROUP_mean_180D', 'SUBGROUP_std_180D', 'SKU_mean_30D', 'SKU_std_30D', 'SKU_mean_90D', 'SKU_std_90D', 'SKU_mean_180D', 'SKU_std_180D', 'STORE_ID_mean_30D', 'STORE_ID_std_30D', 'STORE_ID_mean_90D', 'STORE_ID_std_90D', 'STORE_ID_mean_180D', 'STORE_ID_std_180D']


In [14]:
results_lgb = walk_forward_lightgbm(
    df=datos_unidos,
    features=features,
    target_col="TOTAL_SALES",
    date_col="DATE",
    categorical_cols=cols_categoricas,
    train_days=365,
    step_days=30,
    forecast_days=7,
    params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 2,
        "learning_rate": 0.01,
        "num_leaves": 400,
        "max_depth": 20,
        "min_data_in_leaf": 50,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.9,
        "bagging_freq": 5,
        "early_stopping_round": 30,
        "num_boost_round":1000
    }
)

Walk-forward: iteracion numero 1
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.950365
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.114499
[LightGBM] [Debug] init for col-wise cost 0.067873 seconds, init for row-wise cost 0.428935 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.568096 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6234
[LightGBM] [Info] Number of data points in the train set: 2242841, number of used features: 39
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score 299.483629
[LightGBM] [Debug] Re-bagging, using 2019054 data to train
[LightGBM] [Debug] Trained a tre

In [17]:
results_lgb.to_csv("resultados_test/resultados_lgb6.csv")

In [18]:
results_lgb.mean()

train_end_date    2022-12-27 00:00:00
r2_train                     0.746958
r2_test                      0.692159
dtype: object

## Deploy

### Training

In [20]:
import lightgbm as lgb

In [21]:
cols = list(datos_unidos.columns)
features = [col for col in cols if col not in ["DATE", "TOTAL_SALES",
                                               'INITIAL_TICKET_PRICE', 'BASE_PRICE', "COSTOS", 
                                               "OPENDATE", "CLOSEDATE", "QUANTITY", "STORE_SUBGROUP_DATE_ID"] ]

print(features)

['SKU', 'STORE_ID', 'PRICE', 'REGION', 'CITY', 'STATE', 'STORE_TYPE', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE', 'PRICE_GROUP_ID', 'BRAND', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK', 'SUBGROUP_mean_30D', 'SUBGROUP_std_30D', 'SUBGROUP_mean_90D', 'SUBGROUP_std_90D', 'SUBGROUP_mean_180D', 'SUBGROUP_std_180D', 'SKU_mean_30D', 'SKU_std_30D', 'SKU_mean_90D', 'SKU_std_90D', 'SKU_mean_180D', 'SKU_std_180D', 'STORE_ID_mean_30D', 'STORE_ID_std_30D', 'STORE_ID_mean_90D', 'STORE_ID_std_90D', 'STORE_ID_mean_180D', 'STORE_ID_std_180D']


In [22]:
params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 1,
        "learning_rate": 0.01,
        "num_leaves": 200,
        "max_depth": 20,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.9,
        "bagging_freq": 1,
		"num_boost_round" : 100,
    }

In [23]:
data_train = lgb.Dataset(datos_unidos[features], datos_unidos["TOTAL_SALES"], categorical_feature=cols_categoricas)

In [24]:
model_lgb = lgb.train(params, data_train)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2,747764 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6239
[LightGBM] [Info] Number of data points in the train set: 6839616, number of used features: 40
[LightGBM] [Info] Start training from score 266,144489


### Template dataframe

Primero, creamos un dataframe base con todos los productos y tiendas para los 7 días, de manera que el modelo pueda predecir las ventas de cada combinación

In [ ]:
template = crear_template(datos_unidos)

In [ ]:
# Como el modelo lo entrenaremos con 152 tiendas, pero dos de ellas cerraron y solo haremos la predicción de 150 tiendas, 
# necesitamos ajustar las categorias para que funcione correctamente lgb
type_stores = pd.api.types.CategoricalDtype(categories=datos_unidos["STORE_ID"].unique())
template["STORE_ID"] = template["STORE_ID"].astype(type_stores)

In [ ]:
template = template.sort_values(by=["DATE", "STORE_ID", "SKU"])

In [ ]:
template = rolling_sales_template(df=datos_unidos, template=template, group= "SUBGROUP", windows=[30, 90, 180])
template = rolling_sales_template(df=datos_unidos, template=template, group= "SKU", windows=[30, 90, 180])
template = rolling_sales_template(df=datos_unidos, template=template, group= "STORE_ID", windows=[30, 90, 180])

In [ ]:
template.columns

### Optimizacion de precios

In [ ]:
price_grid = crear_price_grid(datos_unidos, n_prices=10)

In [ ]:
mejor_y_pred, mejor_sales, mejor_gain, mejor_config = optimizacion_precios(template=template, model=model_lgb, price_grid=price_grid,
                                                             features=features, n_iter=10, target="GAIN", 
                                                             save_dir="resultados_optimizacion",
                                                             file_name="mejor_config_lgb")

In [ ]:
mejor_df = pd.read_csv("resultados_optimizacion/mejor_config_lgb.csv")